# Details
I just tried to clip the linear regression prediction -ve values to 0 to submit.
my score got improved from 0.73 to 0.72 

In [65]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
import gc

DATA_FOLDER = '..//data//'
d_parser = lambda x: pd.datetime.strptime(x,'%Y-%m-%d')

import warnings
warnings.filterwarnings("ignore")

In [66]:
def get_rolling_mean(df):
    '''
    this function will create rolling mean on group by id, and sales
    '''
    lags = [1,2,3,4,5,6]
    lag_cols = [f'lag_{lag}' for lag in lags]
    
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[['id','target']].groupby('id')['target'].shift(lag)
    
    wins = [2,4,6]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            df[f'rmean_{lag}_{win}'] = df[['id', lag_col]].groupby('id')[lag_col].transform(lambda x : x.rolling(win).mean())

In [67]:
df_sample_sub = pd.read_csv(os.path.join(DATA_FOLDER,'sample_submission.csv'))
df_ste        = pd.read_csv(os.path.join(DATA_FOLDER,'sales_train_evaluation.csv'))
df_prices     = pd.read_csv(os.path.join(DATA_FOLDER,'sell_prices.csv'))
df_calander   = pd.read_csv(os.path.join(DATA_FOLDER,'calendar.csv'), parse_dates=["date"], date_parser=d_parser)

In [68]:
df_ste_rows = df_ste.melt(
id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
var_name ='d',
value_name ='target'
)
del df_ste

In [69]:
df_ste_rows['d']   = df_ste_rows['d'].apply(lambda x: x.replace('d_',''))
df_ste_rows['d']   = df_ste_rows['d'].astype('int16')

In [70]:
def get_d_of_m(col): 
    return col.day

In [71]:
df_calander['d'] = df_calander['d'].apply(lambda x: x.replace('d_',''))
df_calander['d'] = df_calander['d'].astype('int16')
gc.collect()

#get day of month
df_calander['dom'] = df_calander['date'].apply(get_d_of_m)

#dropping the date column
df_calander.drop(['date'], axis=1, inplace=True)

In [72]:
df_calander.head()

,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,dom
0,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0,29
1,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0,30
2,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0,31
3,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0,1
4,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1,2


In [73]:
#merging sales with calendar
calendar_merge_cols = ['wm_yr_wk','wday','d','month','year','dom']
df_ste_rows = df_ste_rows.merge(df_calander[calendar_merge_cols], on=['d'], how='left')

#merging sales with prices
df_ste_rows = df_ste_rows.merge(df_prices, how='left', on=['store_id','item_id','wm_yr_wk'])

#type casting the resulting columns
df_ste_rows.wm_yr_wk     = df_ste_rows.wm_yr_wk.astype('int16')
df_ste_rows.wday         = df_ste_rows.wday.astype('int8')
df_ste_rows.month        = df_ste_rows.month.astype('int8')
df_ste_rows.dom          = df_ste_rows.dom.astype('int8')
df_ste_rows.year         = df_ste_rows.year.astype('int16')
df_ste_rows.sell_price   = df_ste_rows.sell_price.astype('float16')

#deleting the dataframes not required
del df_calander, df_prices
gc.collect()

0

In [74]:
df_ste_rows =df_ste_rows[(df_ste_rows.d > (df_ste_rows.d.max() - 180))]

In [75]:
get_rolling_mean(df_ste_rows)

In [76]:
df_ste_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5488200 entries, 53692890 to 59181089
Data columns (total 38 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          object 
 1   item_id     object 
 2   dept_id     object 
 3   cat_id      object 
 4   store_id    object 
 5   state_id    object 
 6   d           int16  
 7   target      int64  
 8   wm_yr_wk    int16  
 9   wday        int8   
 10  month       int8   
 11  year        int16  
 12  dom         int8   
 13  sell_price  float16
 14  lag_1       float64
 15  lag_2       float64
 16  lag_3       float64
 17  lag_4       float64
 18  lag_5       float64
 19  lag_6       float64
 20  rmean_1_2   float64
 21  rmean_2_2   float64
 22  rmean_3_2   float64
 23  rmean_4_2   float64
 24  rmean_5_2   float64
 25  rmean_6_2   float64
 26  rmean_1_4   float64
 27  rmean_2_4   float64
 28  rmean_3_4   float64
 29  rmean_4_4   float64
 30  rmean_5_4   float64
 31  rmean_6_4   float64
 32  rmean_1_6   float64
 33 

In [81]:
df_ste_rows.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,target,wm_yr_wk,wday,...,rmean_3_4,rmean_4_4,rmean_5_4,rmean_6_4,rmean_1_6,rmean_2_6,rmean_3_6,rmean_4_6,rmean_5_6,rmean_6_6
54028280,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1773,0,11545,2,...,0.75,1.25,1.25,1.25,0.500000,0.833333,1.000000,1.000000,0.833333,0.833333
54028281,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1773,1,11545,2,...,0.00,0.25,0.25,0.25,0.333333,0.166667,0.166667,0.166667,0.166667,0.166667
54028282,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1773,0,11545,2,...,0.50,1.25,1.50,1.75,0.333333,0.833333,1.166667,1.500000,1.500000,1.833333
54028283,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1773,2,11545,2,...,2.25,3.00,2.25,2.75,2.500000,2.666667,2.500000,2.666667,2.166667,2.666667
54028284,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1773,0,11545,2,...,0.75,0.75,0.75,0.50,1.000000,0.666667,0.500000,0.500000,0.500000,0.333333


In [80]:
df_ste_rows.dropna(inplace=True)

In [106]:
drop_cols = input('enter :')

enter :


In [107]:
len(drop_cols)

0

In [105]:
drop_cols = drop_cols.split(',')
drop_cols 

['one', ' trwo', ' three']

In [79]:
df_ste_rows.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'target', 'wm_yr_wk', 'wday', 'month', 'year', 'dom', 'sell_price',
       'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'rmean_1_2',
       'rmean_2_2', 'rmean_3_2', 'rmean_4_2', 'rmean_5_2', 'rmean_6_2',
       'rmean_1_4', 'rmean_2_4', 'rmean_3_4', 'rmean_4_4', 'rmean_5_4',
       'rmean_6_4', 'rmean_1_6', 'rmean_2_6', 'rmean_3_6', 'rmean_4_6',
       'rmean_5_6', 'rmean_6_6'],
      dtype='object')

In [64]:
df_ste_rows[(df_ste_rows.item_id=='HOBBIES_1_001') 
            & (df_ste_rows.store_id=='CA_1')][['rmean_1_1', 'rmean_2_1', 'rmean_3_1',
       'rmean_1_2', 'rmean_2_2', 'rmean_3_2', 'rmean_1_3', 'rmean_2_3',
       'rmean_3_3']]

,rmean_1_1,rmean_2_1,rmean_3_1,rmean_1_2,rmean_2_2,rmean_3_2,rmean_1_3,rmean_2_3,rmean_3_3
53845340,2.0,1.0,0.0,1.5,0.5,0.0,1.000000,0.333333,0.000000
53875830,2.0,2.0,1.0,2.0,1.5,0.5,1.666667,1.000000,0.333333
53906320,0.0,2.0,2.0,1.0,2.0,1.5,1.333333,1.666667,1.000000
53936810,1.0,0.0,2.0,0.5,1.0,2.0,1.000000,1.333333,1.666667
53967300,0.0,1.0,0.0,0.5,0.5,1.0,0.333333,1.000000,1.333333
...,...,...,...,...,...,...,...,...,...
59028640,0.0,0.0,0.0,0.0,0.0,2.0,0.000000,1.333333,2.000000
59059130,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.333333
59089620,3.0,0.0,0.0,1.5,0.0,0.0,1.000000,0.000000,0.000000
59120110,3.0,3.0,0.0,3.0,1.5,0.0,2.000000,1.000000,0.000000


In [41]:
df_ste_rows[(df_ste_rows.item_id=='HOBBIES_1_001') 
            & (df_ste_rows.store_id=='CA_1')
            & (df_ste_rows.d==1823)][['target','lag_7','lag_28']]

,target,lag_7,lag_28
55552780,0,0.0,0.0


In [42]:
df_ste_rows[(df_ste_rows.item_id=='HOBBIES_1_001') 
            & (df_ste_rows.store_id=='CA_1')
            & (df_ste_rows.d==1824)][['target','lag_7','lag_28']]

,target,lag_7,lag_28
55583270,1,2.0,2.0


In [108]:
t = []
s = ['one','two']

u = s+t

In [109]:
u

['one', 'two']